## First create separate images from each wsi image, with annotations in the unstained folder. crop the images (total of 24 annotation, so 24 images per whole slide) and save them first in a different folder in same path. Then pad the images and then save them. Then using those padded images, register them to one another (doesn't matter which way since there are only two images)
## the path is in: \\shelter\Kyu\unstain2stain\biomax_images

In [1]:
import numpy as np
import time
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import cv2
import os
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
from skimage.transform import warp
from skimage.registration import optical_flow_tvl1, optical_flow_ilk
import openslide
import os
import cv2
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import skimage.measure
import scipy.stats as stats
import torchvision.transforms as transforms
from glob import glob
from time import time
from skimage.measure import label


In [2]:
def _time(f):
    def wrapper(*args,**kwargs):
        start=time()
        r=f(*args,**kwargs)
        end=time()
        print("%s timed %f" %(f.__name__,end-start))
        return r
    return wrapper

In [3]:
# First, input xml_filepath and output a dataframe of X,Y coordinates in general. (can be used for ROI as well)
def xml_to_df(xml_filepath):
    tree = ET.parse(xml_filepath)
    root = tree.getroot()
    append_df = []
    for index, Annotation in enumerate(root.iter("Annotation")):
        for Region in Annotation.iter('Region'):
            x = np.array([Vertex.get('X') for Vertex in Region.iter('Vertex')])
            y = np.array([Vertex.get('Y') for Vertex in Region.iter('Vertex')])
            id = np.array([int(Region.get('Id'))])
            classnames = index + 1
            coord_dict = {"ClassNames": [classnames], "X": [x], "Y": [y], "ID": [id]}
            df = pd.DataFrame(data = coord_dict)
            df.ID = df.ID.astype(int)
            append_df.append(df)
    coord_df = pd.concat(append_df).reset_index(drop=True)
    return(coord_df)

In [4]:
coord_df = xml_to_df(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk244a.xml')
coord_df

,ClassNames,X,Y,ID
0,1,"[66855, 66848, 66848, 66848, 66848, 66848, 668...","[29667, 29667, 29674, 29681, 29688, 29695, 297...",1
1,1,"[64319, 64382, 64395, 64421, 64446, 64472, 644...","[63336, 63399, 63412, 63438, 63451, 63476, 635...",2
2,1,"[78061, 78048, 78048, 78048, 78048, 78048, 780...","[28813, 28813, 28851, 28915, 29005, 29068, 291...",3
3,1,"[66800, 66791, 66781, 66763, 66754, 66735, 667...","[38608, 38608, 38617, 38636, 38654, 38682, 387...",4
4,1,"[66050, 66060, 66078, 66106, 66143, 66180, 661...","[49432, 49422, 49404, 49395, 49367, 49330, 493...",5
5,1,"[78649, 78649, 78649, 78649, 78649, 78649, 786...","[38239, 38252, 38264, 38290, 38328, 38354, 383...",6
6,1,"[77691, 77678, 77665, 77639, 77627, 77601, 775...","[48635, 48635, 48635, 48648, 48661, 48686, 487...",7
7,1,"[78163, 78291, 78432, 78559, 78738, 78879, 790...","[67984, 67984, 67997, 67997, 67997, 67997, 679...",8
8,1,"[89727, 89727, 89713, 89699, 89693, 89679, 896...","[66995, 66954, 66905, 66863, 66821, 66793, 667...",9
9,1,"[88630, 88593, 88574, 88537, 88519, 88500, 884...","[55128, 55054, 54999, 54906, 54851, 54795, 547...",10


In [49]:
coord_df = xml_to_df(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk481.xml')
coord_df

,ClassNames,X,Y,ID
0,1,"[30445, 30422, 30395, 30377, 30354, 30332, 303...","[18504, 18504, 18504, 18504, 18504, 18504, 185...",2
1,1,"[27976, 27966, 27960, 27951, 27945, 27939, 279...","[30439, 30448, 30451, 30457, 30460, 30466, 304...",3
2,1,"[26513, 26522, 26525, 26534, 26540, 26555, 265...","[47636, 47642, 47648, 47660, 47672, 47688, 477...",4
3,1,"[40906, 40884, 40842, 40793, 40763, 40736, 407...","[41073, 41073, 41073, 41076, 41076, 41076, 410...",5
4,1,"[37763, 37767, 37770, 37778, 37778, 37782, 377...","[31952, 31945, 31933, 31926, 31915, 31911, 319...",6
5,1,"[38796, 38796, 38796, 38796, 38796, 38796, 387...","[21983, 21899, 21857, 21815, 21731, 21689, 215...",7
6,1,"[61410, 61410, 61403, 61403, 61403, 61403, 614...","[11941, 11900, 11851, 11802, 11739, 11690, 116...",10
7,1,"[53802, 53753, 53711, 53690, 53676, 53655, 536...","[18936, 18936, 18936, 18936, 18936, 18936, 189...",11
8,1,"[65361, 65428, 65435, 65449, 65477, 65519, 655...","[25976, 26020, 26020, 26020, 26034, 26055, 260...",12
9,1,"[54779, 54787, 54800, 54805, 54813, 54818, 548...","[36448, 36444, 36439, 36435, 36431, 36431, 364...",13


In [50]:
coord_df = xml_to_df(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\skn1001.xml')
coord_df

,ClassNames,X,Y,ID
0,1,"[40870, 40865, 40854, 40843, 40838, 40827, 408...","[56954, 56965, 56970, 56986, 56986, 57003, 570...",1
1,1,"[42225, 42216, 42197, 42188, 42178, 42160, 421...","[63912, 63912, 63912, 63912, 63930, 63930, 639...",2
2,1,"[39642, 39632, 39623, 39623, 39623, 39623, 396...","[78875, 78903, 78921, 78940, 78949, 78958, 789...",3
3,1,"[41121, 41032, 41006, 40968, 40955, 40917, 408...","[84822, 84873, 84898, 84937, 84962, 85026, 850...",4
4,1,"[51861, 51848, 51848, 51836, 51823, 51797, 517...","[61554, 61554, 61541, 61528, 61503, 61465, 614...",6
...,...,...,...,...
95,3,"[135093, 135149, 135204, 135260, 135315, 13535...","[34928, 34947, 34965, 34965, 34965, 34984, 349...",11
96,3,"[123806, 123806, 123799, 123799, 123799, 12379...","[60318, 60325, 60339, 60360, 60381, 60416, 604...",12
97,3,"[124375, 124356, 124356, 124338, 124319, 12430...","[47939, 47939, 47957, 47976, 47976, 48012, 480...",13
98,3,"[123764, 123764, 123764, 123746, 123728, 12372...","[42949, 42931, 42912, 42894, 42838, 42820, 427...",14


### for stained: downsample by 8x to find the bounding box for each tissue image in WSI, and then get those boundingbox coordsinates multipled by 8x again, and use those coordinates to read region with no downsample, but only that region of interest and then crop. (modify read_region function to read at full resolution, but only the bounding box by changing location and size).

### for unstained: use xml coordinates and just find the boudnign box from there and then read region with no downsample and crop

In [5]:
# for unstained:
def unstained_crop(xml_filepath, unstain_ndpi_filepath, general_save_path):
    coord_df = xml_to_df(xml_filepath)
    slide = openslide.open_slide(unstain_ndpi_filepath)
    rgb_dim = slide.dimensions
    print("Dimension of level 0 of the unstained image is: ",rgb_dim)
    slide_level_dim = slide.level_dimensions
    num_levels = len(slide_level_dim)
    print("Number of levels in this unstained image are:",num_levels)
    factors = slide.level_downsamples
    print("For unstained image, each level is downsampled by:",factors)
    tmp = os.path.basename(xml_filepath)
    ndpi_name = os.path.splitext(tmp)[0]

    for idx, row in coord_df.iterrows():
        xx = row.X.astype('int')
        yy = row.Y.astype('int')
        x_min = np.min(xx)
        x_max = np.max(xx)
        y_min = np.min(yy)
        y_max = np.max(yy)
        location = (x_min,y_min)
        dim = (x_max-x_min,y_max-y_min)
        image_resized = slide.read_region(location= location,level=0,size=dim)
        image_resized = np.array(image_resized)
        final_img = Image.fromarray(image_resized)
        save_path = os.path.join(general_save_path,ndpi_name)
        save_path = os.path.join(save_path, ndpi_name + str(idx) + ".png")
        final_img.save(save_path)

    print("Images saved successfully!")

In [6]:
unstained_crop(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk244a.xml', r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk244a.ndpi',r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\images')

Dimension of level 0 of the unstained image is:  (199680, 98560)
Number of levels in this unstained image are: 8
For unstained image, each level is downsampled by: (1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0)
Images saved successfully!


In [7]:
unstained_crop(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk481.xml', r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk481.ndpi',r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\images')

Dimension of level 0 of the unstained image is:  (184320, 98560)
Number of levels in this unstained image are: 8
For unstained image, each level is downsampled by: (1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0)
Images saved successfully!


In [8]:
unstained_crop(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\skn1001.xml', r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\skn1001.ndpi',r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\images')

Dimension of level 0 of the unstained image is:  (192000, 98560)
Number of levels in this unstained image are: 8
For unstained image, each level is downsampled by: (1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0)
Images saved successfully!


### Note to self: Note that the stained_crop function below inputs a binary_mask_path that is a binary mask of a downsampled version of the original ndpi. For my case, as seen in the function below, was 16x downsampled. Write a new function before this to create a binary mask of the downsampled version of the ndpi, which would require removing small holes, etc -> code snippet found in wsi2imcrop and other files.

In [30]:
import cv2
def match_images(list1, list2):
    matches = []
    sift = cv2.SIFT_create()
    for img1 in list1:
        kp1, des1 = sift.detectAndCompute(img1,None)
        best_match = (None, float('inf'))
        for img2 in list2:
            # img2 = img2.astype('uint8')
            kp2, des2 = sift.detectAndCompute(img2,None)
            bf = cv2.BFMatcher()
            matches = bf.knnMatch(des1, des2, k = 2)
            good = []
            for m, n in matches:
                if m.distance < 0.75 * n.distance:
                    good.append([m])
            if len(good) > len(best_match[0]):
                best_match = (img2,good)
        if best_match[0] is not None:
            matches.append((img1,best_match[0]))
            list2.remove(best_match[0])
    return matches

In [31]:
list1 = [_ for _ in os.listdir(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\images\sk244a') if _.endswith(".png")]
list1_path = [os.path.join(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\images\sk244a\\' + x) for x in list1]
list2 = [_ for _ in os.listdir(r'\\shelter\Kyu\unstain2stain\biomax_images\stained\images\sk244ahe') if _.endswith(".png")]
list2_path = [os.path.join(r'\\shelter\Kyu\unstain2stain\biomax_images\stained\images\sk244ahe\\' + x) for x in list2]
img1_list = [cv2.imread(x) for x in list1_path]
img2_list = [cv2.imread(x) for x in list2_path]
match_images(img1_list,img2_list)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\features2d\src\matchers.cpp:860: error: (-215:Assertion failed) trainDescCollection[iIdx].rows < IMGIDX_ONE in function 'cv::BFMatcher::knnMatchImpl'


In [42]:
# for stained:
def stained_crop(stain_ndpi_filepath, general_save_path, binary_mask_path):
    slide = openslide.open_slide(stain_ndpi_filepath)
    tmp = os.path.basename(stain_ndpi_filepath)
    ndpi_name = os.path.splitext(tmp)[0]

# since image_resized is read with downsample, now find the coordinates of the bounding box for each image
# to do that, first create a binary mask (which is already found in binary_image_path and then use label to find bbox of each label (image).

    binary_img = Image.open(binary_mask_path)
    labeledbw = label(np.array(binary_img))

    for id in range(1,np.max(labeledbw)):
        bwtarget = labeledbw == id

        # to know where it is saved
        tmp_path = os.path.join(save_path,"location")
        tmp_path = os.path.join(tmp_path, ndpi_name + str(id - 1) + ".png")
        Image.fromarray(bwtarget).save(tmp_path)

        ind = np.argwhere(bwtarget)
        xx_min = np.min(ind[:,1])
        xx_max = np.max(ind[:,1])
        yy_min = np.min(ind[:,0])
        yy_max = np.max(ind[:,0])
        x_min = round(xx_min*16) #downsample factor used to create binary_mask in binary_mask_path was 16
        x_max = round(xx_max*16)
        y_min = round(yy_min*16)
        y_max = round(yy_max*16)
        location = (x_min,y_min)
        dim = (x_max-x_min,y_max-y_min)
        fin_image = slide.read_region(location= location,level=0,size=dim)
        fin_image = np.array(fin_image)
        save_path = os.path.join(general_save_path,ndpi_name)
        save_path = os.path.join(save_path, ndpi_name + str(id - 1) + ".png")
        Image.fromarray(fin_image).save(save_path)
    print("Images saved successfully!")

In [43]:
stained_crop(r'\\shelter\Kyu\unstain2stain\biomax_images\stained\sk244ahe.ndpi', r'\\shelter\Kyu\unstain2stain\biomax_images\stained\images', r'\\shelter\Kyu\unstain2stain\biomax_images\stained\entire_binary_masks\sk244a.png')

Images saved successfully!


In [44]:
stained_crop(r'\\shelter\Kyu\unstain2stain\biomax_images\stained\sk481he.ndpi', r'\\shelter\Kyu\unstain2stain\biomax_images\stained\images', r'\\shelter\Kyu\unstain2stain\biomax_images\stained\entire_binary_masks\sk481.png')

Images saved successfully!


In [45]:
stained_crop(r'\\shelter\Kyu\unstain2stain\biomax_images\stained\skn1001he.ndpi', r'\\shelter\Kyu\unstain2stain\biomax_images\stained\images', r'\\shelter\Kyu\unstain2stain\biomax_images\stained\entire_binary_masks\skn1001.png')

Images saved successfully!


### Now the problem before padding the two images together is that the order of the images produced from the unstained (using the xml) and the stained (using the skimage.measure.label) method are different. Below code creates the location of each image so that the names can be manually changed by matching the location.

In [ ]:
coord_df = xml_to_df(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk244a.xml')
slide = openslide.open_slide(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk244a.ndpi')
rgb_dim = slide.dimensions
(x_size,y_size) = (rgb_dim[0] / 16 , rgb_dim[1] / 16)
x_size = int(x_size)
y_size = int(y_size)

for idx, row in coord_df.iterrows():
    blank = np.ones((y_size, x_size))
    xx = row.X.astype('int')
    yy = row.Y.astype('int')
    xx = xx/16
    yy = yy/16
    xx = np.round(xx)
    yy = np.round(yy)
    xy = list(zip(xx,yy))
    contours = np.array(xy)
    mask = cv2.fillPoly(blank, pts=[contours.astype(int)], color=idx+125)
    mask_img = Image.fromarray(mask)
    save_path = os.path.join(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained', "location")
    save_path = os.path.join(save_path, "sk244a" + str(idx) + ".png")
    mask_img = mask_img.convert('RGB')
    mask_img.save(save_path)

coord_df = xml_to_df(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk481.xml')
slide = openslide.open_slide(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\sk481.ndpi')
rgb_dim = slide.dimensions
(x_size,y_size) = (rgb_dim[0] / 16 , rgb_dim[1] / 16)
x_size = int(x_size)
y_size = int(y_size)

for idx, row in coord_df.iterrows():
    blank = np.ones((y_size, x_size))
    xx = row.X.astype('int')
    yy = row.Y.astype('int')
    xx = xx/16
    yy = yy/16
    xx = np.round(xx)
    yy = np.round(yy)
    xy = list(zip(xx,yy))
    contours = np.array(xy)
    mask = cv2.fillPoly(blank, pts=[contours.astype(int)], color=idx+125)
    mask_img = Image.fromarray(mask)
    save_path = os.path.join(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained', "location")
    save_path = os.path.join(save_path, "sk481" + str(idx) + ".png")
    mask_img = mask_img.convert('RGB')
    mask_img.save(save_path)

coord_df = xml_to_df(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\skn1001.xml')
slide = openslide.open_slide(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained\skn1001.ndpi')
rgb_dim = slide.dimensions
(x_size,y_size) = (rgb_dim[0] / 16 , rgb_dim[1] / 16)
x_size = int(x_size)
y_size = int(y_size)

for idx, row in coord_df.iterrows():
    blank = np.ones((y_size, x_size))
    xx = row.X.astype('int')
    yy = row.Y.astype('int')
    xx = xx/16
    yy = yy/16
    xx = np.round(xx)
    yy = np.round(yy)
    xy = list(zip(xx,yy))
    contours = np.array(xy)
    mask = cv2.fillPoly(blank, pts=[contours.astype(int)], color=idx+125)
    mask_img = Image.fromarray(mask)
    save_path = os.path.join(r'\\shelter\Kyu\unstain2stain\biomax_images\unstained', "location")
    save_path = os.path.join(save_path, "skn1001" + str(idx) + ".png")
    mask_img = mask_img.convert('RGB')
    mask_img.save(save_path)

# for sk244a:
binary_img = Image.open(r'\\shelter\Kyu\unstain2stain\biomax_images\stained\entire_binary_masks\sk244a.png')
labeledbw = label(np.array(binary_img))

for id in range(1,np.max(labeledbw)):
    bwtarget = labeledbw == id

    # to know where it is saved
    tmp_path = os.path.join(r'\\shelter\Kyu\unstain2stain\biomax_images\stained',"location")
    tmp_path = os.path.join(tmp_path, "sk244a" + str(id - 1) + ".png")
    Image.fromarray(bwtarget).save(tmp_path)

# for sk481:
binary_img = Image.open(r'\\shelter\Kyu\unstain2stain\biomax_images\stained\entire_binary_masks\sk481.png')
labeledbw = label(np.array(binary_img))

for id in range(1,np.max(labeledbw)):
    bwtarget = labeledbw == id

    # to know where it is saved
    tmp_path = os.path.join(r'\\shelter\Kyu\unstain2stain\biomax_images\stained',"location")
    tmp_path = os.path.join(tmp_path, "sk481" + str(id - 1) + ".png")
    Image.fromarray(bwtarget).save(tmp_path)

# for skn1001:
binary_img = Image.open(r'\\shelter\Kyu\unstain2stain\biomax_images\stained\entire_binary_masks\skn1001.png')
labeledbw = label(np.array(binary_img))

for id in range(1,np.max(labeledbw)):
    bwtarget = labeledbw == id

    # to know where it is saved
    tmp_path = os.path.join(r'\\shelter\Kyu\unstain2stain\biomax_images\stained',"location")
    tmp_path = os.path.join(tmp_path, "skn1001" + str(id - 1) + ".png")
    Image.fromarray(bwtarget).save(tmp_path)

In [3]:
# function to pad images to same size:
def pad_images_to_same_size(images):
    """
    :param images: sequence of images
    :return: list of images padded so that all images have same width and height (max width and height are used)
    """
    width_max = 0
    height_max = 0
    for img in images:
        h, w = img.shape[:2]
        width_max = max(width_max, w)
        height_max = max(height_max, h)

    images_padded = []
    for img in images:
        h, w = img.shape[:2]
        diff_vert = height_max - h
        pad_top = diff_vert//2
        pad_bottom = diff_vert - pad_top
        diff_hori = width_max - w
        pad_left = diff_hori//2
        pad_right = diff_hori - pad_left
        img_padded = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=(255,255,255))
        assert img_padded.shape[:2] == (height_max, width_max)
        images_padded.append(img_padded)

    return images_padded

In [4]:
pad_images_to_same_size = _time(pad_images_to_same_size)
optical_flow_tvl1 = _time(optical_flow_tvl1)

In [ ]:
# pad the two images:


### Now register the two padded images:

In [ ]:
def registrate_two_images(reference_image_path, moving_image_path, save_path):
    """
    Note: The order of the files saved in the ref_img_path and mov_img_path must be the same so that you are registering the same images!
    """
    ref_img_path = [_ for _ in os.listdir(reference_image_path) if _.endswith(".png")]
    ref_img_path_complete = [os.path.join(ref_img_path, x) for x in ref_img_path]
    ref_img_path_1 = [x.replace('.png','') for x in ref_img_path]

    mov_img_path = [_ for _ in os.listdir(moving_image_path) if _.endswith(".png")]
    # mov_img_path_complete = [os.path.join(mov_img_path, x) for x in mov_img_path]
    # mov_img_path_1 = [x.replace('.png','') for x in mov_img_path]

    if int(len(ref_img_path)) != int(len(mov_img_path)):
        print("Number of images in reference and moving file paths are not equal, please fix and try again!")
        return

    num = int(len(ref_img_path))
    start = time()
    for idx in range(num,len(ref_img_path_complete)):
        if idx == len(ref_img_path_complete) - 1:
            break
        ref_img = np.array(Image.open(ref_img_path))
        mov_img = np.array(Image.open(mov_img_path))
        ref_img_g = rgb2gray(ref_img)
        mov_img_g = rgb2gray(mov_img)
        v, u = optical_flow_tvl1(ref_img_g, mov_img_g)
        nr, nc = ref_img_g.shape
        row_coords, col_coords = np.meshgrid(np.arange(nr), np.arange(nc),
                                             indexing='ij')
        mov_img_warp_ra =[]
        for i in range(3):
            mov_img_warp = warp(mov_img[:,:,i], np.array([row_coords + v, col_coords + u]),mode='edge')
            mov_img_warp_ra.append(mov_img_warp)
        r = np.array(mov_img_warp_ra[0]*255).astype('uint8')
        g = np.array(mov_img_warp_ra[1]*255).astype('uint8')
        b = np.array(mov_img_warp_ra[2]*255).astype('uint8')
        rgb = np.stack([r,g,b],axis=2)
        reg_img = Image.fromarray(rgb)
        reg_img.save(save_path + str(ref_img_path_1[idx]) + '.png')

    end = time()
    print("time it took to register: "+  str((end-start)/60) + " minutes")
